In [6]:
import os
import re
import csv

def get_corresponding_file(query_file, unified_folder):
    # Strip _query or _query[number] from the filename
    base_name = re.sub(r'_query\d*', '', query_file)
    
    # Search for the corresponding file in the UNIFIED folder
    for file in os.listdir(unified_folder):
        if file.startswith(base_name):
            return file
    
    return None

def strip_extension(filename):
    return os.path.splitext(filename)[0]

def write_to_tsv(query_folder, unified_folder, output_tsv):
    query_files = os.listdir(query_folder)
    
    # Sort the query files alphabetically
    query_files.sort()
    
    with open(output_tsv, mode='w', newline='') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t')
        
        # Write header row
        writer.writerow(['query', 'groundtruth'])
        
        for query_file in query_files:
            if query_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.tif')):
                corresponding_file = get_corresponding_file(query_file, unified_folder)
                if corresponding_file:
                    query_name = strip_extension(query_file)
                    groundtruth_name = strip_extension(corresponding_file)
                    writer.writerow([query_name, groundtruth_name])

# Define the folders and output file path
query_folder = '../data/test_suite/imgs'
unified_folder = '../data/unified/imgs'
output_tsv = 'GT_absolute.tsv'

write_to_tsv(query_folder, unified_folder, output_tsv)


In [ ]:
import pandas as pd
from IPython.display import display_html, HTML

def display_images_with_text_below(df):
    html = """<table border='1' style='border-collapse: collapse; text-align: center;'>
              <tr><th>Query Image</th><th>Groundtruth Image</th></tr>"""

    for _, row in df.iterrows():
        html += "<tr>"

        # Query image and text below
        html += f"""
            <td>
                <img src="{row['image']}" style="width:300px;"><br>
                <div style="padding: 4px; word-break: break-word;">{row['query']}</div>
            </td>
        """

        # Groundtruth image and text below
        html += f"""
            <td>
                <img src="{row['image_original']}" style="width:300px;"><br>
                <div style="padding: 4px; word-break: break-word;">{row['groundtruth']}</div>
            </td>
        """

        html += "</tr>"

    html += "</table>"
    display_html(HTML(html))

# Load the TSV
df = pd.read_csv('GT_absolute.tsv', sep='\t')

# Construct full image paths
df['image'] = '../data/test_suite/imgs/' + df['query'] + '.png'
df['image_original'] = '../data/unified/imgs/' + df['groundtruth'] + '.png'

# Display
display_images_with_text_below(df)
